
# 1. Introduction

This notebook has been developed for the purposes of the EMPIR project “Metrology for the Factory of the Future” (Met4FoF) -  Activity A1.2.2 of the Work Package 1.Two laboratories in PTB and CEM posses conventional dynamic calibration set-ups for acceleration sensors under test. The objective was to extend current calibration systems for digital-output sensors.  The objective of the task is to compare measurement results of the two laboratories, PTB and CEM, provided by the extended dynamic calibration systems. 

## 1.1. State of the art

In conventional dynamic calibration procedures for acceleration sensors, the acceleration used for the sensor input is applied either as **a sinusoidal excitation with a given frequency and amplitude** or as **a singular shock-like excitation characterized by pulse width and intensity**. 
<br>The quantity is then measured by a reference sensor and by the device under test (DUT). The results of DUT are compared to the reference and hence characterized and linked to the SI. In a dynamic calibration situation, the response of the DUT to time-varying input is the major interest. Hence, it is crucial that the mechanical input operates simultaneously and equally on the reference and DUT. Both, the reference and the DUT provide **electrical outputs (typical voltage)** while **the data acquisition electronics** of the system provides **the analogue to digital conversion (ADC).**
In order to connect the reference and the DUT to the ADC system, two analogue channels are needed. In order to get reliable information on the time dependent response, the timing of the data acquisition on the two channels has to be synchronized. This is typically accomplished by provision of a common clock signal to drive the sampling units of the ADC and a common trigger to start (or mark) the beginning of the acquisition. 

## 1.2. The extension of conventional dynamic calibration systems to digital-output sensors 

In a set-up where the DUT is a digital-output sensor,the sampling of the DUT time series is no longer under the control (trigger, clock) of the calibration system. Instead, the DUT comprises its own digitizer unit with a time base independent of the calibration system. In fact, a typical conventional calibration set-up does not provide an input for digital data at all.
The solution to this problem requires two extensions:
1.	A digital acquisition unit (DAU) which is capable to connect to the digital interface of the sensor under calibration and store/transmit the DUT time-series for later analysis
2.	An additional synchronization signal that provides the link between the time-base of the calibration system and the time-base of the DUT.

The concept for the extension of existing facilities for dynamic calibration uses a custom digital acquisition unit microcontroller board with a connected reference time signal for traceable time stamping of acquired data points. This allows for a synchronised data acquisition from the reference measurement and the DUT. The selected sensor for acceleration measurements is a three axial low-g acceleration sensor with digital output, which allows measurements of acceleration in three perpendicular axes.**At the moment and in this notebook, only an X-axis acceleration has been observed.** 
    
The selected laboratories posses their set-ups and the calibration items were calibrated in both set-ups.The analysis required transformation from time domain to the frequency domain, where frequencies and corresponding magnitudes and phases were calculated.The measurement conditions were kept according to the laboratory standard conditions. 

Each laboratory submitted HDFT files containing groups:
- EXPERIMENTS - containing 171 files (experiment) corresponding to the sine excitations
- RAWDATA - where data from ADCs (*voltage, absolute time, absolute time uncertainty*)  and sensors (*absolute time, absolute time uncertainty, acceleration, angular velocity, magnetic flux density and temperature*) can be approached during the measurements
- RAWTRANSFERFUNCTION - *this group contains quantities of interest: frequencies, amplitudes, assigned uncertainties of amplitudes, phases,  assigned uncertainties of phases, excitation amplitudes and assigned uncertainties of excitation amplitudes
- REFERENCEDATA - reference data from ADCs and sensors.

    
| <b>PTB<b> | Sensor | Internal ADC |
| --- | --- | --- |
| <b>Name<b> | MPU 9250| STM 32 Internal ADC |
| <b>ID<b>  | 535035904 | 535038464 |
| <b>Quantity<b>  | X Acceleration | Voltage  @CH1 |
| <b>Unit<b>  | $\frac{m}{s^{2}}$ | V |    
| <b>Resolution<b>  | 65536,0 (16-bit) | 4096,0 (12-bit) |
| <b>Min. scale<b>  | -156,91439819335938 $\frac{m}{s^{2}}$  | -10 V |
| <b>Min. scale<b>  | 156,90960693359375 $\frac{m}{s^{2}}$  | 10 V |
    
| <b>CEM<b> | Sensor | Internal ADC |
| --- | --- | --- |
| <b>Name<b> | MPU 9250| STM 32 Internal ADC |
| <b>ID<b>  | 3167420416 | 31674422976 |
| <b>Quantity<b>  | X Acceleration | Voltage  @CH1 |
| <b>Unit<b>  | $\frac{m}{s^{2}}$ | V |    
| <b>Resolution<b>  | 65536,0 (16-bit) | 4096,0 (12-bit) |
| <b>Min. scale<b>  | -156,91439819335938 $\frac{m}{s^{2}}$  | -10 V |
| <b>Min. scale<b>  | 156,90960693359375 $\frac{m}{s^{2}}$  | 10 V |

In [ ]:
import h5py
import numpy as np
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
import scipy.stats as stats
import seaborn as sns

# 2.Extract the data

Data for ILC comparison is extracted from the HDF5 files separately for PTB and CEM. The extracted data will be sorted by frequency at the end of the Notebook and saved into Excel file.

In [ ]:
def extract_data(filename, sensor_index):
    #explore the HDF5 file, folders and subfolders
    with h5py.File(filename,'r') as f:
        base_items=list(f.items())
        print("\nItems in directory", base_items)
        rawtransfer=f.get("RAWTRANSFERFUNCTION")
        rawtransfer_items=list(rawtransfer.items())
        print("\nItems in reference", rawtransfer_items)
        subgroup=rawtransfer.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250")
        subgroup_items=list(subgroup.items())
        print("\n"+sensor_index+"_MPU_9250 items:",subgroup_items)
        subgroup_acceleration=subgroup.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration")
        subgroup_acceleration_items=list(subgroup_acceleration.items())
        print("\nAcceleration items:",subgroup_acceleration_items)
        subgroup_acceleration_5mem=subgroup.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration")
        subgroup_acceleration_5mem_items=list(subgroup_acceleration_5mem.items())
        print("\nAcceleration items_5members:", subgroup_acceleration_5mem_items)
        frequency=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Frequency")
        frequency_items=list(frequency.items())
        print("\nFrequency", frequency_items)
        magnitude=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Magnitude")
        magnitude_items=list(magnitude.items())
        print("\nMagnitude", magnitude_items)
        phase=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Phase")
        phase_items=list(magnitude.items())
        print("\nPhase", phase_items)
        
        
        #extract frequencies, magnitude, phase, uncertainties and all excitation parameters
        frequency_values=np.array(frequency.get("value"))
        magnitude_values=np.array(magnitude.get("value"))
        magnitude_uncertainties=np.array(magnitude.get("uncertainty"))
        phase_values=np.array(phase.get("value"))
        phase_uncertainties=np.array(phase.get("uncertainty"))
        excitation_freq_items=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Excitation_frequency")
        excitation_freq=np.array(excitation_freq_items.get("value"))
        excitation_amp_items=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Excitation_amplitude")
        excitation_amp=np.array(excitation_amp_items.get("value"))
        excitation_amp_uncertainty=np.array(excitation_amp_items.get("uncertainty"))
        
        #join all necessary data in 2D array
        total_array=np.stack((frequency_values,magnitude_values,magnitude_uncertainties,phase_values, phase_uncertainties,excitation_freq,excitation_amp,excitation_amp_uncertainty), axis=1)
        print("\nArray dimensions:", total_array.shape)
        column_names=["Frequency [Hz]", "Magnitude [m s^-2/m s^-2 ]","Uncertainty [m s^-2/m s^-2 ]", "Phase [°]", "Uncertainty[°]","Excitation_freq [Hz]","Excitation_amplitude [m s^-2]","Excitation_amplitude_uncert [m s^-2]"]
        whole_dataset=pd.DataFrame(total_array, columns=column_names)
        f.close()
        
        
        return whole_dataset

In [ ]:
whole_dataset_PTB = extract_data('MPU9250PTB_v2.hdf5',"0x1fe40000")
#whole_dataset_PTB = extract_data('MPU9250PTB.hdf5',"0x1fe40000")

In [ ]:
whole_dataset_PTB.head(2)

Phase data for PTB must be reverted:

In [ ]:
whole_dataset_PTB[['Phase [°]']] = whole_dataset_PTB[['Phase [°]']]-np.pi
whole_dataset_PTB.head(2)

In [ ]:
whole_dataset_CEM = extract_data('MPU9250CEM_v2.hdf5',"0xbccb0000")
#whole_dataset_CEM = extract_data('MPU9250CEM.hdf5',"0xbccb0000")

In [ ]:
whole_dataset_CEM.head(2)

In [ ]:
#whole_dataset_CEM[['Phase [°]']] = whole_dataset_CEM[['Phase [°]']]-np.pi
#whole_dataset_CEM.head(2)

# 3.Data analysis

In [ ]:
def plot_dataset(dataset,name):
    plt.figure(figsize=(15,10))
    plt.subplot(311)
    plt.plot(dataset['Magnitude [m s^-2/m s^-2 ]'])
    plt.ylabel('Magnitude [m s^-2/m s^-2 ]')
    plt.xlabel("Experiments")
    plt.title(name)

    plt.subplot(312)
    plt.plot(dataset['Phase [°]'])
    plt.ylabel('Phase [°]')
    plt.xlabel("Experiments")

    plt.subplot(313)
    plt.plot(dataset['Excitation_amplitude [m s^-2]'])
    plt.ylabel('Excitation_amplitude [m s^-2]')
    plt.xlabel("Experiments")
    plt.show()

In [ ]:
plot_dataset(whole_dataset_PTB,"PTB")

In [ ]:
plot_dataset(whole_dataset_CEM,"CEM")
# peaks are individual behaviour, the system at CEM works in a way that starts with the freq of 80 Hz.

Cycles in CEM's dataset start with 80.0 Hz and 250.0 Hz instead of 10.0 Hz. These starting points are deleted in order to compare the cycles in a range from 10.0 Hz and 250.Hz.

In [ ]:
delete_rows=[]

for k in range(0,171,19):
    i=k
    j=k+1
    delete_rows.append(i)
    delete_rows.append(j)
whole_dataset_CEM_new=whole_dataset_CEM.drop(axis=0,index=delete_rows)

In [ ]:
plot_dataset(whole_dataset_CEM_new,"CEM corrected")


In [ ]:
#digital sensors have integrated temp sensor. Phase does not suffer the high influence of the temperature. High influence of the temperature
#is visible for amplitude. PLOT amplitude, samp.freq and temp. Look at the sine fit of the temp.

In [ ]:
def plot_magnitude_and_phase(dataset,name):  
   
    f,ax=plt.subplots(nrows=2,ncols=2,figsize=(20,15))
    
    x_axis=np.where(dataset['Frequency [Hz]'] == 10.0)

    for j,i in enumerate(x_axis[0]):
        x=dataset["Frequency [Hz]"].values[int(i):int(i)+17]
        y1=dataset['Magnitude [m s^-2/m s^-2 ]'].values[int(i):int(i)+17]
        y2=dataset['Phase [°]'].values[int(i):int(i)+17]
       
    #linear axis
        sns.scatterplot(x=x,y=y1,ax=ax[0,0], label=j+1)
        ax[0,0].set_xlabel("Frequency [Hz]",fontsize=16)
        ax[0,0].set_ylabel("Magnitude [m s^-2/m s^-2 ]",fontsize=16)
        ax[0,0].set_title(name,fontsize=16)

        sns.scatterplot(x=x,y=y2,ax=ax[1,0],label=j+1)
        ax[1,0].set_xlabel("Frequency [Hz]",fontsize=16)
        ax[1,0].set_ylabel("Phase [°]",fontsize=16)

        #log x-axis
        sns.scatterplot(x=x,y=y1,ax=ax[0,1],label=j+1)
        ax[0,1].set_xscale("log")
        ax[0,1].set_ylabel('Magnitude [m s^-2/m s^-2 ]',fontsize=16)
        ax[0,1].set_xlabel("log(Frequency [Hz])",fontsize=16)
        ax[0,1].set_title(name,fontsize=16)
                        
        sns.scatterplot(x=x,y=y2,ax=ax[1,1],label=j+1)
        ax[1,1].set_xscale("log")
        ax[1,1].set_ylabel('Phase [°]',fontsize=16)
        ax[1,1].set_xlabel("log(Frequency [Hz])",fontsize=16)


    plt.show()

In [ ]:
plot_magnitude_and_phase(whole_dataset_PTB,"PTB")

In [ ]:
plot_magnitude_and_phase(whole_dataset_CEM_new,"CEM corrected")

In [ ]:
def split_data_by_frequencies(dataset):
    dict_of_frequencies=dict(iter(dataset.groupby('Frequency [Hz]')))
    return dict_of_frequencies
    #list_of_frequencies=np.array([10,12.5,16,20,25,31.5,40,46.7,50,53.3,63,80,100,125,160,200,250])

In [ ]:
#check if all frequencies are the same
PTB_separated_by_freq=split_data_by_frequencies(whole_dataset_PTB)
CEM_separated_by_freq=split_data_by_frequencies(whole_dataset_CEM)
CEM_separated_by_freq_new=split_data_by_frequencies(whole_dataset_CEM_new)
print("Frequencies - PTB:",PTB_separated_by_freq.keys())
print("Frequencies - CEM:",CEM_separated_by_freq.keys())

In [ ]:
PTB_separated_by_freq.get(10).head(1)

In [ ]:
CEM_separated_by_freq.get(10).head(1)

In [ ]:
q_names=list(PTB_separated_by_freq.get(10).columns)

The interval of  $\bar {x_{i}} -\sigma _{i}$ and $ \bar {x_{i}} +\sigma _{i}$  is plotted for each experiment and the quantities of interest.

In [ ]:
def plot_x_plus_minus_sigma(dictionary,names):
    for val in dictionary.values():
    
        data_dict_amp = {}
        data_dict_amp['cycle'] =list(range(len(val)))
        data_dict_amp['lower'] = val["Magnitude [m s^-2/m s^-2 ]"].values-val["Uncertainty [m s^-2/m s^-2 ]"].values
        data_dict_amp['upper'] = val["Magnitude [m s^-2/m s^-2 ]"].values+val["Uncertainty [m s^-2/m s^-2 ]"].values
        dataset_amp = pd.DataFrame(data_dict_amp)

        data_dict_phase = {}
        data_dict_phase['cycle'] =list(range(len(val)))
        data_dict_phase['lower'] = val["Phase [°]"].values-val["Uncertainty[°]"].values
        data_dict_phase['upper'] = val["Phase [°]"].values+val["Uncertainty[°]"].values
        dataset_phase = pd.DataFrame(data_dict_phase)

        data_dict_excit = {}
        data_dict_excit['cycle'] =list(range(len(val)))
        data_dict_excit['lower'] = val["Excitation_amplitude [m s^-2]"].values-val["Excitation_amplitude_uncert [m s^-2]"].values
        data_dict_excit['upper'] = val["Excitation_amplitude [m s^-2]"].values+val["Excitation_amplitude_uncert [m s^-2]"].values
        dataset_excit = pd.DataFrame(data_dict_excit)


        plt.figure(figsize=(15,5))
        plt.subplot(1,2,1)

        for lower,upper,y in zip(dataset_amp['lower'],dataset_amp['upper'],range(len(dataset_amp))):
            plt.plot((lower,upper),(y,y),'ro-',color='orange')
        plt.yticks(range(len(dataset_amp)),list(dataset_amp['cycle']))
        plt.title(str(val[names[0]].values[0])+" Hz")
        plt.xlabel("Magnitude")
        plt.ylabel("Measurement cycle")

        plt.subplot(1,2,2)
        for lower,upper,y in zip(dataset_phase['lower'],dataset_phase['upper'],range(len(dataset_phase))):
            plt.plot((lower,upper),(y,y),'ro-',color='orange')
        plt.yticks(range(len(dataset_phase)),list(dataset_phase['cycle']))
        plt.title(str(val[names[0]].values[0])+" Hz")
        plt.xlabel("Phase")
        plt.ylabel("Measurement cycle")

        #plt.subplot(1,3,3)
        #for lower,upper,y in zip(dataset_excit['lower'],dataset_excit['upper'],range(len(dataset_excit))):
            #plt.plot((lower,upper),(y,y),'ro-',color='orange')
        #plt.yticks(range(len(dataset_excit)),list(dataset_excit['cycle']))
        #plt.xlabel("Excitation amplitude")
        #plt.ylabel("Measurement cycle")
        #plt.title(str(val[names[0]].values[0])+" Hz")
    plt.show()

In [ ]:
plot_x_plus_minus_sigma(PTB_separated_by_freq,q_names)
# At low frequencies, excit amplitudes are very low. It is hard to target the excitation amplitude (usually varies 1%)
#the sampling frequency starts form 10 Hz becausethe  shaker in calibration system should be bigger in order th achieve 0 Hz.
#AT ptb high unc for phases at low freq

In [ ]:
plot_x_plus_minus_sigma(CEM_separated_by_freq_new,q_names)

In [ ]:
for val1,val2 in zip(PTB_separated_by_freq.values(),CEM_separated_by_freq_new.values()):  
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
 
    x = np.linspace(0,len(val1[q_names[1]]),len(val1[q_names[1]]))
    dy = val1[q_names[2]].values
    y = val1[q_names[1]].values
    plt.xlabel("Measurement cycle")
    plt.ylabel("Magnitude")
    plt.errorbar(x, y, yerr=dy, fmt='.k',ecolor='r',label="PTB")
    plt.title(str(val1[q_names[0]].values[0])+" Hz")
    
    x1 = np.linspace(0,len(val2[q_names[1]]),len(val2[q_names[1]]))
    plt.errorbar(x1,y=val2[q_names[1]].values,yerr =val2[q_names[2]].values,fmt='ro',ecolor='g',label="CEM")
    plt.legend()

    plt.subplot(1,2,2)
    x = np.linspace(0,len(val1[q_names[3]]),len(val1[q_names[3]]))
    dy = val1[q_names[4]].values
    y = val1[q_names[3]].values
    plt.xlabel("Measurement cycle")
    plt.ylabel("Phase")
    plt.errorbar(x, y, yerr=dy, fmt='.k',ecolor='r',label="PTB")
                          
    x1 = np.linspace(0,len(val2[q_names[3]]),len(val2[q_names[3]]))
    plt.errorbar(x1,y=val2[q_names[3]].values,yerr =val2[q_names[4]].values,fmt='ro',ecolor='g',label="CEM")
    plt.legend()
    
    #plt.subplot(1,3,3)
    #x = np.linspace(0,len(val1[q_names[6]]),len(val1[q_names[6]]))
    #dy = val1[q_names[7]].values
    #y = val1[q_names[6]].values
    #plt.xlabel("Measurement cycle")
    #plt.ylabel("Excitation amplitude")
    #plt.errorbar(x, y, yerr=dy, fmt='.k',ecolor='r',label="PTB")
                          
    #x1 = np.linspace(0,len(val2[q_names[6]]),len(val2[q_names[6]]))
    #plt.errorbar(x1,y=val2[q_names[6]].values,yerr =val2[q_names[7]].values,fmt='ro',ecolor='b',label="CEM")
    plt.legend()
    plt.show()
    
    #the phase  - its timing information is not very susceptible, similar for PTB and CEM 
#Magnitude  - The difference between CEM and PTB is not the consequence of the different calibration systems. 
#sensor is differently behaving in Spain due to the influence of the tempeerature. Also, it is cheap and the axis can be different 
#cosine error

#Magnitude - manufacturer specifies 1% of tranfer coefficient deviation. Normally, higher uncertainty is stated because 
#due to the mounting influence. (statistically + extraunc)

## 3.2 Coefficient of variation

The coefficient of variation (CV) is a statistical measure of the dispersion of data points in a data series around the mean. The coefficient of variation represents the ratio of the standard deviation to the mean, and it is a useful statistic for comparing the degree of variation from one data series to another, even if the means are drastically different from one another.[1]

For each experiment, CV in percentages is calculated as:
$$CV={\frac {\sigma _{i}}{\bar {x_{i}}}} *100 $$
,where x represents magnitude, phase, and excitation amplituded and $\sigma$ corresponds to the assigned uncertainty values.

In [ ]:
for val1,val2 in zip(PTB_separated_by_freq.values(),CEM_separated_by_freq_new.values()):  
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
 
    x = np.linspace(0,len(val1[q_names[1]]),len(val1[q_names[1]]))
    y_coef = (val1[q_names[2]].values/np.abs(val1[q_names[1]].values))*100
    plt.xlabel("Measurement cycle")
    plt.ylabel("Variation coefficient - Magnitude,%")
    plt.plot(x, y_coef,color='r',label="PTB")
    plt.title(str(val1[q_names[0]].values[0])+" Hz")
    x1 = np.linspace(0,len(val2[q_names[1]]),len(val2[q_names[1]])) 
    plt.plot(x1,(val2[q_names[2]].values/np.abs(val2[q_names[1]].values))*100,color='g',label="CEM")
    plt.legend()

    plt.subplot(1,2,2)
    x = np.linspace(0,len(val1[q_names[3]]),len(val1[q_names[3]]))
    y = (val1[q_names[4]].values/np.abs(val1[q_names[3]].values))*100
    plt.xlabel("Measurement cycle")
    plt.ylabel("Variation coefficient - Phase,%")
    plt.plot(x, y,color='r',label="PTB")
    x1 = np.linspace(0,len(val2[q_names[4]]),len(val2[q_names[4]]))                      
    plt.plot(x1,(val2[q_names[4]].values/np.abs(val2[q_names[3]].values))*100,color='g',label="CEM")
    plt.legend()
    
    #plt.subplot(1,3,3)
    #x = np.linspace(0,len(val1[q_names[6]]),len(val1[q_names[6]]))
    #y = (val1[q_names[7]].values/np.abs(val1[q_names[6]].values))*100
    #plt.xlabel("Measurement cycle")
    #plt.ylabel("Variation coefficient - Excitation amplitude,%")
    #plt.plot(x, y, color='r',label="PTB")
                          
    #x1 = np.linspace(0,len(val2[q_names[6]]),len(val2[q_names[6]]))
    #plt.plot(x1,(val2[q_names[7]].values/np.abs(val2[q_names[6]].values))*100,color='b',label="CEM")
    plt.show()
    

Mean variation coefficients are calculated as averages of all CVs at a given frequency:

$${\bar {CV}={\frac{\sum _{i=1}^{n}CV _{i}}{n}}}$$
,where n represents the number of experiments at a specific frequency.

In [ ]:
def calculate_mean_variation_coefficient(dictionary):
    variation_coef=np.empty((len(dictionary),3))
    for val1,i in zip(dictionary.values(),range(len(dictionary))):  
        variation_coef[i,0]=((val1[q_names[2]].values/np.abs(val1[q_names[1]].values))*100).mean()
        variation_coef[i,1]=((val1[q_names[4]].values/np.abs(val1[q_names[3]].values))*100).mean()
        variation_coef[i,2]=((val1[q_names[7]].values/np.abs(val1[q_names[6]].values))*100).mean()
    return variation_coef

In [ ]:
variation_coef_PTB=calculate_mean_variation_coefficient((PTB_separated_by_freq))
variation_coef_CEM=calculate_mean_variation_coefficient((CEM_separated_by_freq_new))

In [ ]:
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    list_of_freq=[10,12.5,16,20,25,31.5,40,46.7,50,53.3,63,80,100,125,160,200,250]
    x = np.asarray(list_of_freq)
    y_coef = variation_coef_PTB[:,0]
    
    plt.xlabel("Frequencies, Hz")
    plt.ylabel("Variation coefficient - Magnitude,%")
    plt.plot(x, y_coef,color='r',label="PTB")
    plt.plot(x,variation_coef_CEM[:,0],color='g',label="CEM")
    plt.legend()

    plt.subplot(1,3,2)
    x = np.asarray(list_of_freq)
    plt.xlabel("Frequencies, Hz")
    plt.ylabel("Variation coefficient - Phase,%")
    plt.plot(x, variation_coef_PTB[:,1],color='r',label="PTB")
    plt.plot(x,variation_coef_CEM[:,1],color='g',label="CEM")
    plt.legend()
    
    #plt.subplot(1,3,3)
    #x = np.asarray(list_of_freq)
    #plt.xlabel("Frequencies, Hz")
    #plt.ylabel("Variation coefficient - Excitation amplitude,%")
    #plt.plot(x, variation_coef_PTB[:,2], color='r',label="PTB")
    #plt.plot(x, variation_coef_CEM[:,2],color='b',label="CEM")
    #plt.legend() 
    #plt.show()
    

# 8. Datasets for the Excel validation

In [ ]:
#this is for analysis in Excel
def sort_values(dataset):
    dataset=dataset.sort_values("Frequency [Hz]", ascending=True)
    return dataset

In [ ]:
whole_dataset_PTB=sort_values(whole_dataset_PTB)
whole_dataset_CEM=sort_values(whole_dataset_CEM_new)

In [ ]:
whole_dataset_PTB.head(2)

In [ ]:
whole_dataset_CEM.head(2)

In [ ]:
whole_dataset_PTB["Lab"]="PTB"
whole_dataset_CEM["Lab"]="CEM"

In [ ]:
 result = pd.concat([whole_dataset_PTB,whole_dataset_CEM])

In [ ]:
f,ax=plt.subplots(nrows=2,ncols=1,figsize=(15,8))
sns.scatterplot(x="Frequency [Hz]", y="Magnitude [m s^-2/m s^-2 ]", hue="Lab",ax=ax[0],
                    data=result) #palette="Set2")
sns.scatterplot(x="Frequency [Hz]", y="Magnitude [m s^-2/m s^-2 ]", hue="Lab",ax=ax[1],
                    data=result )
ax[1].set_xscale("log")
ax[1].set_xlabel("log (Frequency [Hz])")
plt.show()

In [ ]:
f,ax=plt.subplots(nrows=2,ncols=1,figsize=(15,8))
sns.scatterplot(x="Frequency [Hz]", y="Phase [°]", hue="Lab",ax=ax[0],
                    data=result) #palette="Set2")
sns.scatterplot(x="Frequency [Hz]", y="Phase [°]", hue="Lab",ax=ax[1],
                    data=result )
ax[1].set_xscale("log")
ax[1].set_xlabel("log (Frequency [Hz])")
plt.show()

In [ ]:
list_freq=result["Frequency [Hz]"].unique()
f,ax=plt.subplots(nrows=len(list_freq),ncols=2,figsize=(15,100))
for i,val in enumerate(list_freq):
    
    result1=result.where(result["Frequency [Hz]"]==val)
    sns.violinplot(x="Frequency [Hz]", y="Magnitude [m s^-2/m s^-2 ]", hue="Lab",ax=ax[i,0],
                    data=result1, palette="muted", split=False,orient="v")
    sns.violinplot(x="Frequency [Hz]", y="Phase [°]", hue="Lab",ax=ax[i,1],
                    data=result1, palette="muted", split=False,orient="v")

In [ ]:
list_freq=result["Frequency [Hz]"].unique()

for i,val in enumerate(list_freq):
    
    result1=result.where(result["Frequency [Hz]"]==val)
    result2=result1[['Frequency [Hz]', 'Magnitude [m s^-2/m s^-2 ]','Phase [°]',"Lab"]]
    sns.pairplot(result2, hue="Lab")
    

In [ ]:
result2=result[['Frequency [Hz]', 'Magnitude [m s^-2/m s^-2 ]','Phase [°]',"Lab"]]
sns.pairplot(result2, hue="Lab")

In [ ]:
writer2 = pd.ExcelWriter('All_datasets.xlsx')

whole_dataset_CEM.to_excel(writer2, sheet_name = 'CEM', index = False)
whole_dataset_PTB.to_excel(writer2, sheet_name = 'PTB', index = False)

writer2.save()

# References

[1] https://www.investopedia.com/terms/c/coefficientofvariation.asp
<br>[2] https://en.wikipedia.org/wiki/Weighted_arithmetic_mean
<br>[3] https://en.wikipedia.org/wiki/Effective_sample_size
<br>[4] https://online.stat.psu.edu/stat500/lesson/10/10.2/10.2.1
<br>[5] https://www.marsja.se/levenes-bartletts-test-of-equality-homogeneity-of-variance-in-python/